In [21]:
% matplotlib inline
import os
import shutil
import math
import rasterio
import pandas as pd
import geopandas as gpd
from collections import defaultdict
from shapely.geometry import Polygon
from pprint import pprint

In [2]:
workdir = os.getcwd()
rasterdir = os.path.join("gdal_mask", "raster")
rfile = "Land_Use.tif"

In [3]:
with rasterio.open(os.path.join(rasterdir, rfile), 'r') as srcrast:
    rcrs = srcrast.crs
    rbounds = srcrast.bounds
    rny, rnx = srcrast.shape
    rdy, rdx = srcrast.res

In [4]:
vdx, vdy = 3000, 3000
minx, miny, maxx, maxy = rbounds.left-vdx, rbounds.bottom-vdy, rbounds.right+vdx, rbounds.top+vdy

In [5]:
vnx = int(math.ceil(abs(maxx - minx)/vdx))
vny = int(math.ceil(abs(maxy - miny)/vdy))

In [7]:
polyid = 0
temp_dict = defaultdict(list)
for i in range(vny):
    for j in range(vnx):
        polyid += 1
        vertices = []
        vertices.append((min(minx+vdx*j,maxx),max(maxy-vdy*i,miny)))
        vertices.append((min(minx+vdx*(j+1),maxx),max(maxy-vdy*i,miny)))
        vertices.append((min(minx+vdx*(j+1),maxx),max(maxy-vdy*(i+1),miny)))
        vertices.append((min(minx+vdx*j,maxx),max(maxy-vdy*(i+1),miny)))
        temp_dict['id'].append(polyid)
        temp_dict['geometry'].append(Polygon(vertices))

In [31]:
gdf = gpd.GeoDataFrame(temp_dict, geometry='geometry')
gdf.crs = rcrs.to_string()

In [35]:
outdir = "fishnet"
outpath = os.path.join(workdir, "gdal_mask", outdir)
if os.path.exists(outpath):
    shutil.rmtree(outpath)
os.makedirs(outpath)
outshpfile = "rect_fishnet.shp"
outshpfpath = os.path.join(outpath, outshpfile)
gdf.to_file(outshpfpath)

In [33]:
outshpfpath

'/home/serenewizard/foss/gdal_mask/fishnet/fishnet'